In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
from dash_table.Format import Format, Group, Scheme, Symbol
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

#username and password
username = "aacuser"
password = "ACCUSER"
shelter = AnimalShelter(username, password)


# assining the functions that allow for sorting located in the AnimalShelter PY file
df = pd.DataFrame.from_records(shelter.read({}))
dfwr = pd.DataFrame.from_records(shelter.RescueTypeWater())
dfmwr = pd.DataFrame.from_records(shelter.RescueTypeMountainWilderness())
dfdit = pd.DataFrame.from_records(shelter.RescueTypeDisasterTracking())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#logo File
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())
                         ,style={'height':'25%', 'width':'25%'})),
    #Centering and scaling the logo for display
    html.Center(html.B(html.H1('Michelle Hitchcock'))), #name/key
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),

    html.Div(className='row',
         style={'display' : 'flex'},
             children = [        
                 #Radio buttons + filter values 
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WAT'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOU'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIS'},
                {'label': 'Reset', 'value':'RES'}
            ],
            value='R',
            labelStyle={'display': 'inline-block'}
            ),
 
                  
                      
    ]),
    html.Hr(),
    dt.DataTable( #data table for filtering
        id='datatable-id',
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}
            
            
        ],
        data=df.to_dict('records'),
        #interactive features
 
        filter_action="native", #adds a search bar 
        sort_action="native", #ads a sort feature
        sort_mode="multi", #multi sorting
        row_selectable="single", #allows for selection
        page_action = 'native', #reduces page size
        page_current = 0,
        page_size = 5, 
 
    style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
    ]
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            ) 

        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# filtering using the functions created in the py file.
        if filter_type == 'RES':
            new_df = df
        elif filter_type == 'WAT':
            new_df = dfwr
        elif filter_type == 'MOU':
            new_df = dfmwr
        elif filter_type == 'DIS':
            new_df = dfdit
        else:
            new_df = df
            
        data = new_df.to_dict('records')
        #using the same data table to allow for filtering/retriving of the results
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}  
        ]
        
        return (data, columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns): #coloring selected columbs
    return [{
        'if': { 'column_id': i },
        'background_color': '#d4064b' #similar color as logo
    } for i in selected_columns]
    


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData): #pi charge function
        dff = pd.DataFrame.from_dict(viewData) 
        new_df = dff.breed.value_counts() #the value is based on the breed
        new_df.name = "breed" #giving it a name
        new_df = new_df.to_frame()
        new_df.reset_index(inplace=True) #making sure it updates
        
        
        return [
        dcc.Graph(
        figure = px.pie(new_df, values="breed", names="index") #generating the chart
        )
        ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id',"derived_viewport_selected_rows")])
def update_map(viewData, row): #map function
        odff = df if viewData is None else pd.DataFrame.from_dict(viewData) #if there is no input nothing shows
        if row is None: #when there is input this checks what the input is
            dff = odff #and assigns based on the row that is selected
        else:
            dff = odff.iloc[row,:]
#generating the map
        return [ dl.Map(style={'width': '750px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children = [
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0,9])
                    ])
                ])
            ])
        ]


app